In [ ]:
import csv
import pandas as pd
df = pd.read_csv('/content/Ar_PuFi.csv')
new_lexicon = pd.read_csv('/content/New_Lexicon.csv')
old_lexicon = pd.read_csv('/content/Old_Lexicon.csv')

Offensive_comments = [comment for idx, comment in enumerate(df['Text']) 
               if df['Label'][idx] == '0']

other_comments = [comment for idx, comment in enumerate(df['Text']) if df['Label'][idx] != '0']

def evaluation(predictedHateComments, Offensive_comments, other_tweets):
    FalsePositive = 0 
    FalseNegative = 0
    for comment in predictedHateComments:
        if not comment in Offensive_comments:
            FalsePositive+=1
    TruePositive = len(predictedHateComments) - FalsePositive
    
    for comment in Offensive_comments:
        if not comment in predictedHateComments:
            FalseNegative+=1
    TrueNegative = len(other_comments) - FalsePositive
    
    accuracy = float(TruePositive + TrueNegative)/float(TruePositive + FalsePositive + TrueNegative + FalseNegative)
    precision = float(TruePositive)/float(TruePositive + FalsePositive)
    recall = float(TruePositive)/float(TruePositive + FalseNegative)
    f1_score = 2*((precision*recall)/(precision+recall))
    
    return accuracy, precision, recall, f1_score

old_predictedHateComments = []
for comment in df['Text']:
    for word in old_lexicon['clean']:
        if word in str(comment):
            old_predictedHateComments.append(comment)
            break
accuracy, precision, recall, f1_score = evaluation(old_predictedHateComments, Offensive_comments, other_comments)
print ("accuracy : %.4f, precision : %.4f, recall : %.4f, f1_score : %.4f" %(accuracy, precision,recall,f1_score))

predictedHateComments = []
for comment in df['Text']:
    for word in new_lexicon['clean']:
        if word in str(comment):
            predictedHateComments.append(comment)
            break
accuracy, precision, recall, f1_score = evaluation(predictedHateComments, Offensive_comments, other_comments)
print ("accuracy : %.4f, precision : %.4f, recall : %.4f, f1_score : %.4f" %(accuracy, precision,recall,f1_score))
import nltk

dataset_words = []
for comment in df['Text']:
    for word in str(comment).split(" "):
        dataset_words.append(word)
len(dataset_words)

Offensive_Language_words = []
for comment in Offensive_comments:
    for word in str(comment).split(" "):
        Offensive_Language_words.append(word)

all_fdist = nltk.FreqDist(word for word in dataset_words)
offensive_fdist = nltk.FreqDist(word for word in Offensive_Language_words)
import matplotlib.pyplot as plt

all_fdist.plot(30)

offensive_fdist.plot(30)

old_lex_dist = {}
for word in set(old_lexicon['clean']):
    old_lex_dist[word] = offensive_fdist[word]

old_lex_tuple = [(value, key) for key, value in old_lex_dist.items()]
old_lex_tuple.sort(key=lambda tup: tup[0], reverse=True)
old_lex_tuple

new_lex_dist ={}
for word in set(new_lexicon['clean']):
    new_lex_dist[word] = offensive_fdist[word]

new_lex_tuple = [(value, key) for key, value in new_lex_dist.items()]
new_lex_tuple.sort(key=lambda tup: tup[0], reverse=True)
new_lex_tuple
plt.plot(list(old_lex_dist.values()))
plt.plot(list(new_lex_dist.values()))

old_word_cat = dict()
for cat in set(old_lexicon['category']):
    old_word_cat[cat]= 0
for idx, word in enumerate(old_lexicon['clean']): 
    old_word_cat[old_lexicon['category'][idx]]+= offensive_fdist[word]
old_catCount = [(cat, count) for cat,count in old_word_cat.items()]
old_catCount.sort(key=lambda tup: tup[1], reverse=True)
old_catCount

new_word_cat = dict()
for cat in set(new_lexicon['category']):
    new_word_cat[cat]= 0
for idx, word in enumerate(new_lexicon['clean']): 
    new_word_cat[new_lexicon['category'][idx]]+= offensive_fdist[word]
new_catCount = [(cat, count) for cat,count in new_word_cat.items()]
new_catCount.sort(key=lambda tup: tup[1], reverse=True)
new_catCount

def hatful_offinsive(Offinsive_comments):
    hatful_offinsive_comments = []
    for comment in set(Offinsive_comments):
        for word in new_lexicon['clean']:
            if word in comment:
                hatful_offinsive_comments.append(comment)
                break
    return hatful_offinsive_comments
hatful_offinsive_comments = hatful_offinsive(Offinsive_comments)
hatful_not_offinsive_comments = set(Offinsive_comments) - set(hatful_offinsive_comments)
hatful_not_offinsive_comments